# Demo 2: Retrieving OpenStreetMap Data

## 1. What is OpenStreetMap? 

[OpenStreetMap (OSM)](https://www.openstreetmap.org/#map=12/46.5535/6.6524&layers=D) is a free, crowd-sourced, open geographic database updated and maintained by a community of volunteers via open collaboration. Launched in 2004, it has grown into one of the most comprehensive and detailed sources of geographic information available



### Key Features of OSM Data: 

 1. Geometries: OSM data consists of geometries such as points, lines, and polygons representing various geographic features like roads, buildings, parks, and more.

 2. Attributes: Each geometry in OSM is associated with a set of attributes, providing additional information about the feature, such as its name, type, and other relevant metadata.

 3. Tagging System: OSM uses a tagging system to organize and classify different types of features. Tags consist of key-value pairs that describe the properties of a feature. For example, a road might be tagged with "highway=residential" to indicate it is a residential street.

 4. Community Contributions: OSM data is created and maintained by a global community of volunteers. Contributors use satellite imagery, GPS devices, and local knowledge to update and improve the map.

### Applications of OSM Data:

 - Urban Planning and Development: OSM data is valuable for urban planners and developers to analyze the built environment, identify infrastructure needs, and plan for future growth.

 - Disaster Response and Humanitarian Aid: OSM is often used in disaster response efforts to map affected areas, assess damage, and coordinate relief efforts.

 - Navigation and Routing: OSM powers numerous navigation and routing applications, providing up-to-date and detailed map data for users around the world

## 2. Using OSMnx to Download OSM Data

(NOTE: To use OSMnx, you first need to install it into your environment. For this demo, we have already installed it. But for the future, you can always install it using `conda install osmnx` or `pip install osmnx`)

Imports for the demo: 

In [ ]:
# Imports

# Pour la visualisation
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

%matplotlib inline
import contextily as cx

# Pour manipuler des données géospatiales
import geopandas as gpd
from shapely.geometry import Polygon, box

import pandas as pd


# Pour récupérer des données OpenStreetMap
import osmnx as ox

### 2.1  Download street network data for a specific location

(NOTE: we will tackle this subjet with more detail in the next demo)

In [ ]:
# Define the location (e.g., city name or coordinates)
city_aoi = "Saint Louis Senegal"
# Fetch OSM street network from the location
roads_G = ox.graph_from_place(city_aoi, network_type="all")

# The network_type parameter allows you to specify the type of street network to download
# (e.g., "all", "drive", "bike", "walk").

In [ ]:
# Let's check the which type of data object we donload.
type(roads_G)

A DiGraph is a data type that stores nodes and edges with optional data, or attributes. What we can see here is that this data type belongs to a Python module called networkx that can be used to create, manipulate, and study the structure, dynamics, and functions of complex networks. 


To visualize the graph osmnx has the `plot_graph()`function. `



In [ ]:
# Plot the streets
fig, ax = ox.plot_graph(roads_G)

### 2.2 Exploring the Different Data Layers Available

OpenStreetMap represents physical features on the ground (e.g., buildings, amenities, waterways) using tags attached to its basic data structures (its nodes, ways, and relations). 

Please refer to the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Map_features) to find out more about all the differents map features.

Here is how you can download the features with the `features_from_place()` function.

#### 1. Building  data 

You can check in the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Buildings), all the tags thats you can find with the 'amenity' key.  

In [ ]:
# Download building data
buildings = ox.features_from_place(city_aoi, tags={"building": True})

# What types are those?
print(type(buildings))

As a result we got the data as GeoDataFrames.

In [ ]:
buildings.head()

Get specific type of building :

In [ ]:
hotel = ox.features_from_place(city_aoi, tags={"building": "hotel"})

In [ ]:
hotel

Differece between nodes, way and relation? 

In [ ]:
# Polygons
buildings.loc["way"].explore()

Why do we have nodes in the building data? 

In [ ]:
# Points
buildings.loc["node"].explore()

In [ ]:
# Multipolygons
buildings.loc["relation"].explore()

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
buildings.loc[["way", "relation"]].plot(ax=ax)
ax.set_axis_off()
plt.show()

In [ ]:
# Get the area of the first polygon in the GeoDataFrame
buildings["area"] = buildings["geometry"].area

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
buildings.loc[["way", "relation"]][160:260].plot(
    column="area", ax=ax, legend=True, cmap="Blues"
)
ax.set_axis_off()
plt.show()

#### 2. Amenities (POIS)

You can check in the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Key:amenity), all the tags thats you can find with the 'amenity' key.  

In [ ]:
city2_aoi = "New york, usa"

# Retrieve schools
bars_city = ox.features_from_place(city_aoi, tags={"amenity": "bar"})
bars_city2 = ox.features_from_place(city2_aoi, tags={"amenity": "bar"})

In [ ]:
# How many schools we got?
print("We got", len(bars_city), "bars in", city_aoi)
print("We got", len(bars_city2), "bars in", city2_aoi)

In [ ]:
bars_city2["geometry"]

In [ ]:
# Make subplots that are next to each other
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
fig.suptitle("Comparaison between the data gotten from St.-Louis vs. New York")


# Plot bars in the first city (st-louis)
bars_city.plot(ax=ax1)
# Add title
ax1.set_title("Bars in St-louis")

# Plot bars in the first city (st-louis)
bars_city2.plot(ax=ax2)
# Add title
ax2.set_title("Bars in New York")


# Tight layout often produces nice results
# But requires the title to be spaced accordingly
fig.tight_layout()
fig.subplots_adjust(top=0.88)

plt.show()

We can explore the GeopDataFrame: 

In [ ]:
bars_city2.columns

You can select the columns you are interested in : 

In [ ]:
cols = [
    "name",
    "addr:country",
    "addr:housenumber",
    "addr:postcode",
    "addr:state",
    "addr:street",
    "addr:city",
    "opening_hours",
]

In [ ]:
bars_city2[cols]

Visualize the geoDataFrame: 

In [ ]:
bars_city2.explore()

#### 3. Waterways 


You can check in the [OSM Wiki](https://wiki.openstreetmap.org/wiki/Waterways), all the tags thats you can find with the 'waterway' key.  

In [ ]:
# Fetch waterway data from OpenStreetMap
waterways = ox.features_from_place(city_aoi, tags={"waterway": True})

# Convert the waterways to a GeoDataFrame
waterways_gdf = gpd.GeoDataFrame(waterways)

# Plot the waterways (optional)
waterways_gdf.plot()

### 2.3 From graph to gdf 

The street network data was not in GeoDataFrame format (it was networkx.MultiDiGraph). Luckily, osmnx provides a convenient function `graph_to_gdfs()`` that can convert the graph into two separate GeoDataFrames where the first one contains the information about the nodes and the second one about the edge.

In [ ]:
# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(roads_G)
print("Nodes:\n", nodes.head(), "\n")
print("Edges:\n", edges.head(), "\n")
print("Type:", type(edges))

### 2.4 Plot 

In [ ]:
# load the land use from st.louis
data_filepath = "../data/land_use_gdf_2016.gpkg"
stlouis_land_use_gdf = gpd.read_file(data_filepath)
stlouis_land_use_gdf.head(3)

In [ ]:
# Plot the footprint
ax = stlouis_land_use_gdf.plot(figsize=(10, 10), facecolor="grey")

# Plot the waterways
waterways.to_crs(stlouis_land_use_gdf.crs).plot(ax=ax, facecolor="blue")

# Plot street edges
edges.to_crs(stlouis_land_use_gdf.crs).plot(ax=ax, linewidth=1, edgecolor="#BC8F8F")

# Plot buildings
buildings.to_crs(stlouis_land_use_gdf.crs).plot(ax=ax, facecolor="orange", alpha=0.7)

# Plot restaurants
bars_city.to_crs(stlouis_land_use_gdf.crs).plot(ax=ax, color="yellow", markersize=20)

# Create custom legend handles
legend_handles = [
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        markerfacecolor="yellow",
        markersize=10,
        label="bars",
        alpha=0.7,
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        markerfacecolor="orange",
        markersize=10,
        label="Building Area",
        alpha=0.7,
    ),
    Line2D(
        [0],
        [0],
        marker="o",
        color="w",
        markerfacecolor="blue",
        markersize=10,
        label="Waterways",
        alpha=0.8,
    ),
]

# Add basemap using Contextily
cx.add_basemap(ax, crs=stlouis_land_use_gdf.crs.to_string(), attribution="")

# Add legend with custom handles
ax.legend(handles=legend_handles)

plt.title("POIS in St Louis. 2016")
plt.xlabel("Longitude")
plt.ylabel("Latitude")


plt.show()